In [14]:
import numpy as np
import pandas as pd
import os

In [38]:
data_directory = "../Data/Study/B/26/"
subdirectory = "recalibrated_calibration/"
fileList = os.listdir(data_directory + subdirectory)

In [16]:
def clean_df(df):
    result = df.loc[(df['Movement'] != "start") & (df['Movement'] != "transition") & (df['EyeTrackingEnabled'] == True) & (df['EyeCalibrationValid'] == True) & (df['EyeTrackingEnabledAndValid'] == True) & (df['EyeTrackingDataValid'] == True) & (df['GazeInputSource'] == "Eyes")].copy()
    #result = result[result['Movement'] != "transition"]
    #result = result[result['EyeTrackingDataValid'] == 'TRUE']
    #result = result[result['GazeInputSource'] == 'Eyes']
    result.reset_index(inplace=True, drop=True)
    return result

In [17]:
def calculate_cosine_error(df):
    df['cosineError'] = df.apply(lambda row: np.degrees(np.arccos(np.dot(np.array((row['EyeGazeDirection.x'], row['EyeGazeDirection.y'], row['EyeGazeDirection.z'])), np.array((row['localTransform.position.x'], row['localTransform.position.y'], row['localTransform.position.z'])))/(np.linalg.norm(np.array((row['EyeGazeDirection.x'], row['EyeGazeDirection.y'], row['EyeGazeDirection.z']))) * np.linalg.norm(np.array((row['localTransform.position.x'], row['localTransform.position.y'], row['localTransform.position.z'])))))), axis=1)
    return df

In [18]:
def calculate_euclidean_error(df):
    #df['euclideanError'] = df.apply(lambda row: np.linalg.norm(np.array((row['GazeAngleX'], row['GazeAngleY']))-np.array((row['TargetAngleX'], row['TargetAngleY']))), axis=1)
    #df['recalibratedEuclideanError'] = df.apply(lambda row: np.linalg.norm(np.array((row['CalibratedGazeAngleX'], row['CalibratedGazeAngleY']))-np.array((row['TargetAngleX'], row['TargetAngleY']))), axis=1)
    df['euclideanError'] = np.sqrt((df.GazeAngleX-df.TargetAngleX) * (df.GazeAngleX-df.TargetAngleX) + (df.GazeAngleY-df.TargetAngleY) * (df.GazeAngleY-df.TargetAngleY))
    df['recalibratedEuclideanError'] = np.sqrt((df.CalibratedGazeAngleX-df.TargetAngleX) * (df.CalibratedGazeAngleX-df.TargetAngleX) + (df.CalibratedGazeAngleY-df.TargetAngleY) * (df.CalibratedGazeAngleY-df.TargetAngleY))
    return df

In [20]:
def calculate_errors(output_subdirectory):
    for file in fileList:
        if '.csv' in file:
            df = pd.read_csv(data_directory + subdirectory + file)
            error_df = calculate_euclidean_error(df)
            error_df = calculate_cosine_error(error_df)
            if not os.path.exists(os.path.join(data_directory, output_subdirectory)):
                os.makedirs(os.path.join(data_directory, output_subdirectory))
            error_df_output_path = data_directory + output_subdirectory + '/' + file
            error_df.to_csv(error_df_output_path, index=False)
            print(file)
            print("uncalibrated: " + str(error_df['euclideanError'].mean()))
            print("calibrated: " + str(error_df['recalibratedEuclideanError'].mean()))
            print("cosine: " + str(error_df['cosineError'].mean()))
            print("")

In [39]:
calculate_errors("error/recalibrated_calibration")

wsBodyConstrained_20230515133456_shifted17.csv
uncalibrated: 21.61551152412474
calibrated: 21.162544473975615
cosine: 21.121430899941092

ssWalking_20230515132939_shifted0.csv
uncalibrated: 61.392536835623524
calibrated: 59.60045155823628
cosine: 85.48604196528926

ssWalking_20230515131239_shifted0.csv
uncalibrated: 60.95070382773007
calibrated: 58.99994279048801
cosine: 89.31001080666171

wsWalking_20230515132319_shifted48.csv
uncalibrated: 56.53640267138132
calibrated: 54.42153499870416
cosine: 88.96395304217936

ssHeadConstrained_20230515130934_shifted26.csv
uncalibrated: 6.4474918258856695
calibrated: 1.2552894388206792
cosine: 6.311961048373535

ssHeadConstrained_20230515132735_shifted27.csv
uncalibrated: 5.023500806311865
calibrated: 2.3948584489569873
cosine: 4.9328327723108485

calibration_20230515130828_shifted0.csv
uncalibrated: 6.690758932938728
calibrated: 1.926862930535958
cosine: 6.5300949594798015

hallway_20230515131742_shifted0.csv
uncalibrated: 25.668115993022386
cali